In [115]:
import pandas as pd
import numpy as np

In [116]:
df=pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [117]:
df.isna().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [118]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [119]:
string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].fillna('NA')

In [120]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [121]:
numeric_columns = list(df.dtypes[df.dtypes != 'object'].index)

for col in numeric_columns:
    df[col] = df[col].replace(' ', 0.0)

In [122]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [123]:
df.industry.mode()


0    retail
Name: industry, dtype: object

In [124]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count','converted']


In [125]:
df['interaction_count'].corr(df['lead_score'])

np.float64(0.009888182496913084)

In [126]:
df['annual_income'].corr(df['interaction_count'])

np.float64(0.048618416552580965)

In [127]:
df['number_of_courses_viewed'].corr(df['lead_score'])

np.float64(-0.004878998354681257)

In [128]:
df['number_of_courses_viewed'].corr(df['interaction_count'])

np.float64(-0.023565222882888117)

In [129]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score

In [130]:
df.shape

(1462, 9)

In [131]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [132]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [133]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [134]:
y_train = df_train.lead_score.values
y_val = df_val.lead_score.values
y_test = df_test.lead_score.values

del df_train['lead_score']
del df_val['lead_score']
del df_test['lead_score']

In [140]:
def mutual_info_lead_score(series):
    return mutual_info_score(series, df_full_train.lead_score)

In [141]:
df_full_train[categorical].head

<bound method NDFrame.head of        lead_source       industry employment_status       location
1066  social_media  manufacturing     self_employed      australia
638         events         retail           student  north_america
799   social_media      education                NA         europe
380       referral      education          employed      australia
303       paid_ads     healthcare          employed         europe
...            ...            ...               ...            ...
1130        events  manufacturing          employed  south_america
1294        events     healthcare           student  south_america
860       paid_ads  manufacturing           student  north_america
1459      paid_ads     technology           student  north_america
1126        events  manufacturing     self_employed           asia

[1169 rows x 4 columns]>

In [142]:
mi = df_full_train[categorical].apply(mutual_info_lead_score)
mi.sort_values(ascending=False)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for t

industry             0.345170
location             0.336969
lead_source          0.240912
employment_status    0.205004
dtype: float64

In [145]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression